<a href="https://colab.research.google.com/github/FranklinW10/MLGamePred/blob/main/GamePredUsingPlayerStats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pybaseball
import os
import pandas as pd
import numpy as np
from pybaseball import batting_stats

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.1/426.1 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 15.7 MB/s eta 0:00:00


In [ ]:
from pybaseball import statcast
import pandas as pd
start = "2024-05-16"
end = "2024-05-17"

data = statcast(start_dt=start, end_dt=end)


grouped = data.groupby(['game_date', 'game_pk', 'home_team', 'away_team'])


scores = grouped[['home_score', 'away_score']].max().reset_index()

print(scores)
game_pk = []
game_winner = []
dates = []
for index, row in scores.iterrows():
    dates.append(row['game_date'])
    game_pk.append(row['game_pk'])

    if row['home_score'] > row['away_score']:
        game_winner.append(1)
    else:
        game_winner.append(0)




print(dates)
print(game_pk)
print(game_winner)

This is a large query, it may take a moment to complete


  0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
 50%|█████     | 1/2 [00:01<00:01,  1.01s/it]/usr/local/lib/python3.11/dist-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
100%|██████████| 2/2 [00:04<00:00,  2.04s/it]


    game_date  game_pk home_team away_team  home_score  away_score
0  2024-05-16   745580       PHI       NYM           5           6
1  2024-05-16   745907       MIN       NYY           0           5
2  2024-05-16   746150       LAD       CIN           2           7
3  2024-05-16   746391       HOU       OAK           8           1
4  2024-05-16   746881       CHC       PIT           4           5
5  2024-05-16   746959       BOS        TB           5           7
6  2024-05-17   744934       TOR        TB           3           4
7  2024-05-17   745016       TEX       LAA           3           9
8  2024-05-17   745178       STL       BOS          10           6
9  2024-05-17   745335        SF       COL          10           5
10 2024-05-17   745579       PHI       WSH           4           2
11 2024-05-17   745747       NYY       CWS           4           2
12 2024-05-17   746065       MIA       NYM           8           0
13 2024-05-17   746148       LAD       CIN           7        

In [ ]:
import requests


def get_lineup(game_pk):
    url = f"https://statsapi.mlb.com/api/v1/game/{game_pk}/boxscore"
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Error retrieving game {game_pk}")
        return None

    data = response.json()

    def extract_players(team_data):
        players = []
        for player_id, p in team_data['players'].items():
            if 'battingOrder' in p:
                players.append({
                    'name': p['person']['fullName'],
                    'position': p['position']['abbreviation'],
                    'batting_order': p['battingOrder'],
                    'team': team_data['team']['name']
                })
        return players

    home_lineup = extract_players(data['teams']['home'])
    away_lineup = extract_players(data['teams']['away'])

    def extract_pitchers(team_data):
        pitchers = []
        starter = None

        for player in team_data['players'].values():
            stats = player.get('stats', {})
            pitching = stats.get('pitching', {})
            pos_code = player['position'].get('code')

            if pos_code == '1' and 'pitching' in stats:
                pitcher_info = {
                    'name': player['person']['fullName'],
                    'position': player['position']['abbreviation'],
                    'team': team_data['team']['name'],
                    'games_started': pitching.get('gamesStarted', 0),
                    'is_starter': pitching.get('gamesStarted', 0) == 1
                }

                if pitcher_info['is_starter']:
                    starter = pitcher_info
                else:
                    pitchers.append(pitcher_info)
        if starter:
            return [starter] + pitchers
        return pitchers

    home_pitchers = extract_pitchers(data['teams']['home'])
    away_pitchers = extract_pitchers(data['teams']['away'])

    return home_lineup , away_lineup, home_pitchers, away_pitchers
LineupHome = []
LineupAway = []
PitchersHome = []
PitchersAway = []
for i in range(len(game_pk)):
      game_pk[i] = str(game_pk[i])
      game_curr = game_pk[i]
      homelineup, awaylineup, home_pitchers, away_pitchers = get_lineup(game_curr)
      lineup_df1 = pd.DataFrame(homelineup).sort_values("batting_order").reset_index(drop=True)
      LineupHome.append(lineup_df1)
      pitchers_df1 = pd.DataFrame(home_pitchers)
      PitchersHome.append(pitchers_df1)

      lineup_df2 = pd.DataFrame(awaylineup).sort_values("batting_order").reset_index(drop=True)
      LineupAway.append(lineup_df2)
      pitchers_df2 = pd.DataFrame(away_pitchers)
      PitchersAway.append(pitchers_df2)

print(LineupHome)
print(LineupAway)
print(PitchersHome)
print(PitchersAway)





[                name position batting_order                   team
0     Kyle Schwarber       DH           100  Philadelphia Phillies
1      J.T. Realmuto        C           200  Philadelphia Phillies
2       Bryce Harper       1B           300  Philadelphia Phillies
3          Alec Bohm       3B           400  Philadelphia Phillies
4   Nick Castellanos       RF           500  Philadelphia Phillies
5    Whit Merrifield       LF           600  Philadelphia Phillies
6      Brandon Marsh       CF           700  Philadelphia Phillies
7       Edmundo Sosa       SS           800  Philadelphia Phillies
8       Kody Clemens       2B           801  Philadelphia Phillies
9        Johan Rojas       PR           802  Philadelphia Phillies
10    Cristian Pache       CF           900  Philadelphia Phillies
11      Bryson Stott       SS           901  Philadelphia Phillies,                 name position batting_order             team
0     Edouard Julien       2B           100  Minnesota Twins
1    

In [ ]:
import os
import pandas as pd
from pybaseball import playerid_lookup, statcast_batter, statcast_pitcher
from datetime import datetime, timedelta

def get_player_id(name):
    try:
        last, first = name.split(" ")[-1], " ".join(name.split(" ")[:-1])
        player = playerid_lookup(last, first)
        return int(player.iloc[0]["key_mlbam"])
    except:
        print(f"Couldn't find ID for {name}")
        return None

def save_batter_statcast_csv(player_name, game_date_str, save_dir="player_data"):
    player_id = get_player_id(player_name)
    if not player_id:
        return

    game_date = pd.to_datetime(game_date_str)
    start_date = (game_date - timedelta(days=20)).strftime("%Y-%m-%d")
    end_date = game_date.strftime("%Y-%m-%d")

    try:
        data = statcast_batter(start_dt=start_date, end_dt=end_date, player_id=player_id)
        if not data.empty:
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
            filename = os.path.join(save_dir, f"{player_name.replace(' ', '_')}.csv")
            data.to_csv(filename, index=False)
            print(f" Saved data for {player_name} to {filename}")
        else:
            print(f" No data found for {player_name} between {start_date} and {end_date}")
    except Exception as e:
        print(f" Error downloading data for {player_name}: {e}")

def save_pitcher_statcast_csv(player_name, game_date_str, save_dir="player_data"):
    player_id = get_player_id(player_name)
    if not player_id:
        return
    game_date = pd.to_datetime(game_date_str)
    start_date = (game_date - timedelta(days=20)).strftime("%Y-%m-%d")
    end_date = game_date.strftime("%Y-%m-%d")

    try:
        data = statcast_pitcher(start_dt=start_date, end_dt=end_date, player_id=player_id)
        if not data.empty:
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
            filename = os.path.join(save_dir, f"{player_name.replace(' ', '_')}.csv")
            data.to_csv(filename, index=False)
            print(f" Saved data for {player_name} to {filename}")
        else:
            print(f" No data found for {player_name} between {start_date} and {end_date}")
    except Exception as e:
        print(f" Error downloading data for {player_name}: {e}")

def process_lineup_to_csv(lineup_df1, lineup_df2, pitchers_df1, pitchers_df2, game_date):
    saved_players1 = []
    saved_players2 = []
    saved_pitchers1 = []
    saved_pitchers2 = []
    for _, row in lineup_df1.iterrows():
        player_name = row["name"]
        if player_name not in saved_players1:
            save_batter_statcast_csv(player_name, game_date)
            saved_players1.append(player_name)
    for _, row in lineup_df2.iterrows():
        player_name = row["name"]
        if player_name not in saved_players2:
            save_batter_statcast_csv(player_name, game_date)
            saved_players2.append(player_name)
    for _, row in pitchers_df1.iterrows():
        player_name = row["name"]
        if player_name not in saved_pitchers1:
            save_pitcher_statcast_csv(player_name, game_date)
            saved_pitchers1.append(player_name)
    for _, row in pitchers_df2.iterrows():
        player_name = row["name"]
        if player_name not in saved_pitchers2:
            save_pitcher_statcast_csv(player_name, game_date)
            saved_pitchers2.append(player_name)
    print(saved_players1)
    print(saved_players2)
    print(saved_pitchers1)
    print(saved_pitchers2)


for i in range(len(LineupHome)):
  lineupH = LineupHome[i]
  lineupA = LineupAway[i]
  pitchersH = PitchersHome[i]
  pitchersA = PitchersAway[i]
  start1 = dates[i]
  process_lineup_to_csv(lineupH, lineupA, pitchersH, pitchersA, start1)




Gathering player lookup table. This may take a moment.
Gathering Player Data
 Saved data for Kyle Schwarber to player_data/Kyle_Schwarber.csv
Couldn't find ID for J.T. Realmuto
Gathering Player Data
 Saved data for Bryce Harper to player_data/Bryce_Harper.csv
Gathering Player Data
 Saved data for Alec Bohm to player_data/Alec_Bohm.csv
Gathering Player Data
 Saved data for Nick Castellanos to player_data/Nick_Castellanos.csv
Gathering Player Data
 Saved data for Whit Merrifield to player_data/Whit_Merrifield.csv
Gathering Player Data
 Saved data for Brandon Marsh to player_data/Brandon_Marsh.csv
Gathering Player Data
 Saved data for Edmundo Sosa to player_data/Edmundo_Sosa.csv
Gathering Player Data
 Saved data for Kody Clemens to player_data/Kody_Clemens.csv
Gathering Player Data
 Saved data for Johan Rojas to player_data/Johan_Rojas.csv
Gathering Player Data
 Saved data for Cristian Pache to player_data/Cristian_Pache.csv
Gathering Player Data
 Saved data for Bryson Stott to player_dat

KeyboardInterrupt: 